In [2]:
import Pkg; 

if split(pwd(),"/")[end] == "hmc"
    cd(joinpath(@__DIR__, "../../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using MorphoMolNotebooks
using LinearAlgebra
using Random
using Rotations
using StaticArrays
using Distributions
using Distances
using GLMakie

In [3]:
mol_type = "6r7m"
template_centers = MorphoMol.TEMPLATES[mol_type]["template_centers"]
template_radii = MorphoMol.TEMPLATES[mol_type]["template_radii"]
n_mol = 3
bounds = 70.0
x_init = MorphoMol.get_initial_state(n_mol, bounds)
n_atoms_per_mol = length(template_radii)
radii = vcat([template_radii for i in 1:n_mol]...);

In [15]:
T = 4.0
ε = 0.015
L = 10
β = 1.0 / T

σ_r = 0.75
σ_t = 2.0

rs = 1.4
eta = 0.3665
pf = MorphoMol.Energies.get_prefactors(rs, eta)
overlap_slope = 1.1


Σ = vcat([[σ_r, σ_r, σ_r, σ_t, σ_t, σ_t] for _ in 1:n_mol]...)

energy(x) = MorphoMol.solvation_free_energy_and_measures(x, template_centers, radii, rs, pf, 0.0, overlap_slope, 1.0)
energy_gradient!(∇E, x) = MorphoMol.solvation_free_energy_gradient!(∇E, x, template_centers, radii, rs, pf, overlap_slope)

input = Dict(
    "template_centers" => template_centers,
    "template_radii" => template_radii,
    "n_mol" => n_mol,
    "σ_r" => σ_r,
    "σ_t" => σ_t,
    "rs" => rs,
    "eta" => eta,
    "T" => T,
    "mol_type" => mol_type
)

output = Dict{String, Vector}(
    "states" => Vector{Vector{Float64}}([]),
    "Es" => Vector{Float64}([]), 
    "Vs" => Vector{Float64}([]), 
    "As" => Vector{Float64}([]), 
    "Cs" => Vector{Float64}([]), 
    "Xs" => Vector{Float64}([]),
    "OLs" => Vector{Float64}([]),
    "αs" => Vector{Float32}([]),
)

hmc = MorphoMol.Algorithms.HamiltonianMonteCarlo(energy, energy_gradient!, MorphoMol.Algorithms.standard_leapfrog!, β, L, ε, Σ)
simulation_time_minutes = 10.0
MorphoMol.Algorithms.simulate!(hmc, deepcopy(x_init), simulation_time_minutes, output)
output["αs"][end]

0.73333335f0

In [17]:
println("Initial energy: $(output["Es"][1]) | final energy: $(output["Es"][end]) | minimal energy: $(minimum(output["Es"]))")

Initial energy: 13143.660294476154 | final energy: 13131.28035079803 | minimal energy: 13124.960475995695


In [18]:
try
    rm("assets/output", recursive = true)
catch 
end
mkpath("assets/output")
for (i, state) in enumerate(output["states"])
    MorphoMolNotebooks.configuration_to_poly(MorphoMol.get_flat_realization(state, template_centers), radii, "assets/output/$(i)", n_mol, n_atoms_per_mol)
end

In [19]:
n = length(output["Es"])
e_mindex = argmin(output["Es"])

figure_config = Dict(
    "plot_ms" => 5,
    "tracker_ms" => 10,
    "title_fs" => 20,
    "vis_range" =>  1:n
)

f = Figure(fontsize = 12)
#Slider
sl_i = Slider(f[3, 1:2], range = 1:length(output["states"][figure_config["vis_range"]]), startvalue = 1)
x = sl_i.value
cgl = GridLayout(f[1:2,1:2])
MorphoMolNotebooks.visualize_configuration_sequence!(cgl, x, input, output, figure_config)

display(f)

GLMakie.Screen(...)